# Contract Analysis System - Testing Notebook

## Overview
This notebook provides comprehensive testing for the Contract Analysis System, which includes document reading, text processing, contract validation, and AI-powered analysis using local LLMs (Ollama).

## System Architecture
- **Document Reader**: Handles PDF, DOCX, and text files
- **Text Processor**: Advanced NLP for legal text analysis
- **Contract Validator**: Determines if document is a valid contract
- **LLM Manager**: Unified interface for Ollama, OpenAI, and Anthropic
- **Contract Classifier**: AI-powered contract categorization
- **Model Manager**: Handles model loading and caching


## Import Dependencies

In [2]:
import os
import sys
import json
from pathlib import Path
from pprint import pprint


# Add parent directory to path for module imports
sys.path.append('..')

# Import all system components
from utils.logger import log_info
from utils.logger import log_error
from utils.validators import ContractValidator
from utils.text_processor import TextProcessor
from utils.logger import ContractAnalyzerLogger
from utils.document_reader import DocumentReader
from model_manager.llm_manager import LLMManager
from model_manager.llm_manager import LLMProvider
from model_manager.model_loader import ModelLoader
from services.contract_classifier import ContractClassifier

print("✅ All modules imported successfully!")


✅ All modules imported successfully!


## Configuration

In [3]:
# Configuration settings
CONFIG = {"pdf_file_path"   : "../../../../Downloads/Satyaki Mitra - Employee Agreement - 2021.pdf",
          "use_spacy"       : True,  # Set to False if spaCy not installed
          "ollama_base_url" : "http://localhost:11434",
          "log_directory"   : "contract_analysis_logs",
         }

# Display configuration
print("Configuration Settings:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")
    

Configuration Settings:
  pdf_file_path: ../../../../Downloads/Satyaki Mitra - Employee Agreement - 2021.pdf
  use_spacy: True
  ollama_base_url: http://localhost:11434
  log_directory: contract_analysis_logs


## Document Reader Testing

In [4]:
# Initialize Document Reader
print("📄 STEP 1: Testing Document Reader\n")
print("=" * 60)

def test_document_reader(file_path):
    """
    Test the document reader with various file types
    """
    reader = DocumentReader()
    
    try:
        # Read the document
        file_contents = reader.read_file(file_path_or_bytes = file_path,
                                         file_type          = "pdf",
                                        )
        
        # Extract text content
        if isinstance(file_contents, dict):
            text     = file_contents.get('text', '') or file_contents.get('content', '')
            metadata = {k: v for k, v in file_contents.items() if k != 'text'}
        
        else:
            text     = str(file_contents)
            metadata = dict()
            
        
        # Display results
        print(f"✅ Document read successfully!\n")
        print(f"📊 Text length: {len(text):,} characters\n")
        print(f"\nText preview:\n")
        print("-" * 50)
        print(text)
        print("-" * 50)
        print("\n\n")
        
        if metadata:
            print(f"📋 Metadata: {list(metadata.keys())}")
        
        return text, metadata
        
    except Exception as e:
        print(f"❌ Error reading document: {e}")
        return None, None


# Test with configured PDF file
document_text, document_metadata = test_document_reader(file_path = CONFIG["pdf_file_path"])

if not document_text:
    print("⚠️  No text extracted.")


📄 STEP 1: Testing Document Reader

✅ Document read successfully!

📊 Text length: 26,469 characters


Text preview:

--------------------------------------------------
Itobuz Technologies Pvt. Ltd. Private and Confidential

Agreement of Employment

This Agreement for service (hereinafter referred to as “Agreement”) made and entered into on the 01st day of December 2022, by

and between Itobuz Technologies Private Limited a Company registered under the Companies Act 2013 having registered office at

STEP, IIT KHARAGPUR, P.S.- IIT KHARAGPUR, KHARAGPUR, WEST BENGAL 721302, INDIA, CIN No. U72200WB2010PTC150305

(hereinafter referred to as the “Employer”)

And

Satyaki Mitra son of Debdas Mitra residing at 28/6, Nabin Senapati Lane, P.O. - Baishnab Para Bazaar, P.S. - Shibpur, Howrah,

West Bengal - 711101 (hereinafter referred to as the “Employee”)

RECITALS

A. The Employer is engaged in the business of Software development and Information Technology based services (hereinafter

referred t

## Contract Validation Testing

In [5]:
# Initialize Contract Validator
print("\n🔍 STEP 2: Testing Contract Validator\n")
print("=" * 60)

def test_contract_validation(text):
    """
    Test if the document is a valid contract
    """
    validator = ContractValidator()
    
    print("📋 Running contract validation...")
    
    # Test 1: File integrity check (simulated)
    file_valid, file_message                         = True, "File check simulated - always passes in notebook"
    print(f"📁 File Integrity: {file_valid} - {file_message}\n")
    
    # Test 2: Contract validation
    is_contract, validation_type, validation_message = validator.is_valid_contract(text = text)
    
    print(f"📑 Contract Validation Results:")
    print(f"   Is Contract: {is_contract}")
    print(f"   Confidence: {validation_type}")
    print(f"   Message: {validation_message}\n")
    
    # Test 3: Detailed validation report
    validation_report                                = validator.get_validation_report(text = text)
    
    print(f"\n📊 Detailed Validation Report:")
    print(f"   Total Score: {validation_report['scores']['total']}")
    print(f"   Found Indicators: {len(validation_report['found_indicators'])}")
    print(f"   Anti-patterns: {len(validation_report['found_anti_patterns'])}\n")
    
    # Display key features
    features                                         = validation_report['features']
    print(f"   Key Features:")
    for feature, value in features.items():
        print(f"     - {feature}: {value}")
    
    # Display top indicators
    if validation_report['found_indicators']:
        print(f"   Top Indicators: {validation_report['found_indicators'][:5]}")
    
    return validation_report

# Run validation test
validation_report = test_contract_validation(text = document_text)



🔍 STEP 2: Testing Contract Validator

📋 Running contract validation...
📁 File Integrity: True - File check simulated - always passes in notebook

📑 Contract Validation Results:
   Is Contract: True
   Confidence: high_confidence
   Message: Strong contract indicators detected (score: 95). Found: agreement, contract, party, parties, hereinafter. This is highly likely a legal contract.


📊 Detailed Validation Report:
   Total Score: 88
   Found Indicators: 10
   Anti-patterns: 1

   Key Features:
     - has_signature_block: False
     - has_effective_date: True
     - has_party_identification: True
   Top Indicators: ['agreement', 'contract', 'party', 'parties', 'hereinafter']


## Text Processing Testing

In [6]:
# Initialize Text Processor
print("\n📝 STEP 3: Testing Text Processor")
print("=" * 60)

def test_text_processing(text: str, use_spacy: bool = True):
    """
    Test advanced text processing capabilities
    """
    processor = TextProcessor(use_spacy = use_spacy)
    
    print("🔧 Initializing text processor...")
    
    # Test 1: Basic text statistics
    print("\n📊 1. Text Statistics:")
    text_statistics = processor.get_text_statistics(text = text)
    
    for key, value in text_statistics.items():
        print(f"   {key.replace('_', ' ').title()}: {value}")
    
    # Test 2: Legal entity extraction
    print("\n🏛️  2. Legal Entity Extraction:")
    legal_entities      = processor.extract_legal_entities(text = text)
    legal_entity_counts = {k: len(v) for k, v in legal_entities.items() if v}
    
    for entity_type, count in legal_entity_counts.items():
        print(f"   {entity_type.title()}: {count} found\n")
        
        if ((entity_type in ['parties', 'dates', 'amounts']) and legal_entities[entity_type]):
            # Show first 10 samples
            samples = legal_entities[entity_type][:10]  
            print(f"     Samples: {samples}")
    
    # Test 3: Sentence extraction
    print("\n📝 3. Sentence Analysis:")
    sentences = processor.extract_sentences_advanced(text = text)
    print(f"   Total sentences: {len(sentences)}")
    
    if sentences:
        print("   First 10 sentences with entities:")
        for i, sent in enumerate(sentences[:10]):
            print(f"     {i+1}. {sent['text']}\n")
            if sent['entities']:
                print(f"        Entities: {sent['entities']}")
    
    # Test 4: Text chunking for analysis
    print("\n📦 4. Text Chunking:")
    chunks = processor.chunk_text_for_embedding(text       = text, 
                                                chunk_size = 512, 
                                                overlap    = 50,
                                               )
    
    print(f"   Created {len(chunks)} chunks for analysis\n")
    
    if chunks:
        print(f"   First chunk preview:")
        print(f"     Text: {chunks[0]['text']}\n")
        print(f"     Word count: {chunks[0]['word_count']}\n")
        print(f"     Sentences: {chunks[0]['start_sentence']}-{chunks[0]['end_sentence']}\n\n")
    
    # Test 5: Specialized legal extraction
    print("\n💰 5. Financial & Legal Elements:")
    monetary_amounts = processor.extract_monetary_amounts(text = text)
    durations        = processor.extract_durations(text = text)
    percentages      = processor.extract_percentages(text = text)
    
    print(f"   Monetary amounts: {monetary_amounts}")
    print(f"   Durations: {durations}")
    print(f"   Percentages: {percentages}\n\n")
    
    return {'statistics'     : text_statistics,
            'legal_entities' : legal_entities,
            'sentences'      : sentences,
            'chunks'         : chunks,
           }

# Run text processing test
processing_results = test_text_processing(text      = document_text, 
                                          use_spacy = CONFIG["use_spacy"],
                                         )



📝 STEP 3: Testing Text Processor
[TextProcessor] spaCy model loaded successfully
🔧 Initializing text processor...

📊 1. Text Statistics:
   Character Count: 26469
   Word Count: 4061
   Sentence Count: 168
   Paragraph Count: 264
   Avg Words Per Sentence: 24.172619047619047
   Avg Chars Per Word: 6.517852745629155
   Language: en

🏛️  2. Legal Entity Extraction:
   Parties: 5 found

     Samples: ['Employee', 'Client', 'Employer', 'The Company', 'Itobuz Technologies Private Limited']
   Dates: 1 found

     Samples: ['01-12-2022']
   References: 15 found


📝 3. Sentence Analysis:
   Total sentences: 218
   First 10 sentences with entities:
     1. Itobuz Technologies Pvt.

        Entities: [('Itobuz Technologies Pvt', 'ORG')]
     2. Ltd. Private and Confidential

Agreement of Employment

This Agreement for service (hereinafter referred to as “Agreement”) made and entered into on the 01st day of December 2022, by

and between Itobuz Technologies Private Limited a Company registered 

## LLM Manager Testing (Ollama)

In [7]:
# Initialize LLM Manager
print("\n💬 STEP 4: Testing LLM Manager with Ollama")
print("=" * 60)

def test_llm_manager(text_snippet: str):
    """
    Test LLM capabilities using Ollama
    """
    print("🚀 Initializing LLM Manager...")
    
    try:
        # Initialize LLM manager
        llm_manager         = LLMManager(default_provider = LLMProvider.OLLAMA,
                                         ollama_base_url  = CONFIG["ollama_base_url"],
                                        )
        
        # Check available providers
        available_providers = llm_manager.get_available_providers()
        
        print(f"✅ Available LLM Providers: {[p.value for p in available_providers]}")
        
        if LLMProvider.OLLAMA not in available_providers:
            print("❌ Ollama not available. Please ensure Ollama is running.")
            print("   Start Ollama: ollama serve")
            return None
        
        # Check available models
        ollama_models = llm_manager.list_ollama_models()
        print(f"📚 Available Ollama Models: {ollama_models}")
        
        if not ollama_models:
            print("⚠️  No Ollama models found. Pull a model: ollama pull llama2")
            return None
        
        # Test 1: Basic completion
        print("\n🧪 1. Testing Basic Completion:")
        test_prompt = "What are the key elements of an employment agreement?"
        
        response    = llm_manager.complete(prompt      = test_prompt,
                                           provider    = LLMProvider.OLLAMA,
                                           temperature = 0.1,
                                           max_tokens  = 512,
                                          )
        
        print(f"   Prompt: {test_prompt}")
        print(f"   Response: {response.text}")
        print(f"   Success: {response.success}")
        print(f"   Tokens: {response.tokens_used}")
        print(f"   Latency: {response.latency_seconds:.2f}s")
        
        # Test 2: Contract analysis
        print("\n📑 2. Testing Contract Analysis:")
        analysis_prompt  = f"""
                                Analyze this contract text and identify:
                                1. The parties involved
                                2. Main obligations  
                                3. Key financial terms
                                4. Duration/term
                                
                                Contract text: {text_snippet[:2000]}  # Limit to first 2000 chars
                            """
        
        analysis_response = llm_manager.complete(prompt      = analysis_prompt,
                                                 provider    = LLMProvider.OLLAMA,
                                                 temperature = 0.1,
                                                 max_tokens  = 500,
                                                )
        
        print("\n   Contract Analysis Results:")
        print("   " + "=" * 40)
        print(analysis_response.text)
        print("   " + "=" * 40)
        
        # Test 3: FIXED JSON structured output
        print("\n📋 3. Testing Structured Output:")
        try:
            # Create a better prompt with the actual contract text
            json_prompt      = f"""
                                    Extract the key parties and their roles from the following contract text. Return ONLY valid JSON with no additional text.
                                
                                    Contract Text:
                                    {text_snippet[:1500]}
                                
                                    Return JSON format:
                                    {{
                                      "parties": [
                                        {{
                                          "name": "party_name",
                                          "role": "party_role",
                                          "type": "individual/organization"
                                        }}
                                      ],
                                       "agreement_type": "type_of_agreement",
                                       "effective_date": "date_if_mentioned"
                                    }}
                                """
            
            # Use a more detailed schema description
            schema_description = """
                                    JSON schema with:
                                    - parties: array of objects with name, role, and type
                                    - agreement_type: string describing the type of agreement
                                    - effective_date: string with the effective date if mentioned
                                    - compensation: object with salary/amount details if mentioned
                                 """
            
            json_response      = llm_manager.generate_structured_json(prompt             = json_prompt,
                                                                      schema_description = schema_description,
                                                                      provider           = LLMProvider.OLLAMA,
                                                                      max_tokens         = 1024,
                                                                      temperature        = 0.1,
                                                                     )
            
            print("   Structured JSON Output:")
            pprint(json_response)
            
            # Validate the response
            if ((json_response.get('parties') == ['Alice', 'Bob']) or (json_response.get('roles') == ['Seller', 'Buyer'])):
                print("\n   ⚠️  WARNING: Model generated generic placeholder data!")
                print("   This indicates the model didn't properly analyze the contract.")
                
        except Exception as e:
            print(f"   JSON generation failed: {e}")
            
            # Fallback: Try manual JSON parsing with a simpler approach
            print("\n   🔧 Trying alternative JSON extraction...")
            try:
                fallback_prompt  = f"""
                                        Based on this contract text, extract the parties and their roles in JSON format:
                                        
                                        {text_snippet[:1000]}
                                        
                                        Return ONLY JSON, no other text. Example format:
                                        {{
                                          "parties": [
                                            {{
                                              "name": "Company Name", 
                                              "role": "Employer"
                                            }},
                                            {{
                                              "name": "Employee Name", 
                                              "role": "Employee" 
                                            }}
                                          ]
                                        }}
                                    """
                
                fallback_response = llm_manager.complete(prompt      = fallback_prompt,
                                                         provider    = LLMProvider.OLLAMA,
                                                         temperature = 0.1,
                                                         max_tokens  = 500,
                                                         json_mode   = True,
                                                        )
                
                if fallback_response.success:
                    # Try to parse the response as JSON
                    try:
                        # Clean the response
                        json_text   = fallback_response.text.strip()
                        json_text   = json_text.replace('```json', '').replace('```', '').strip()
                        
                        parsed_json = json.loads(json_text)
                        print("   Alternative JSON Output:")
                        pprint(parsed_json)
                        
                    except json.JSONDecodeError:
                        print("   Could not parse JSON from response:")
                        print(f"   Response: {fallback_response.text}")
                        
            except Exception as fallback_error:
                print(f"   Alternative approach also failed: {fallback_error}")
        
        return llm_manager
        
    except Exception as e:
        print(f"❌ LLM Manager test failed: {e}")
        import traceback
        print(f"Detailed error: {traceback.format_exc()}")
        return None


# Run LLM test with the extracted document text
llm_manager = test_llm_manager(text_snippet = document_text)



💬 STEP 4: Testing LLM Manager with Ollama
🚀 Initializing LLM Manager...
[Logger] Logging initialized. Logs: logs
✅ Available LLM Providers: ['ollama']
📚 Available Ollama Models: ['llama3:8b', 'mistral:7b', 'deepseek-r1:32b', 'qwen3:32b']

🧪 1. Testing Basic Completion:
   Prompt: What are the key elements of an employment agreement?
   Response:  An employment agreement, also known as an offer letter or contract of employment, is a legally binding document that outlines the terms and conditions of employment between an employer and an employee. Here are some key elements typically included in an employment agreement:

1. Parties involved: The names of the employer and employee, along with their respective addresses and contact information.

2. Position and duties: A clear description of the job title, responsibilities, and any specific duties or expectations related to the position.

3. Compensation: Details about the salary, wage, commission structure, bonuses, and any other forms of

## Contract Classification Testing

In [8]:
# Initialize Contract Classifier
print("\n🏷️ STEP 5: Testing Contract Classifier")
print("=" * 60)

def test_contract_classification(text):
    """
    Test AI-powered contract classification
    """
    print("🎯 Initializing Contract Classifier...")
    
    try:
        # Initialize model loader and classifier
        model_loader   = ModelLoader()
        classifier     = ContractClassifier(model_loader)
        
        print("✅ Models loaded successfully!")
        
        # Test 1: Single category classification
        print("\n🔍 1. Single Category Classification:")
        classification = classifier.classify_contract(contract_text = text)
        
        print(f"   Primary Category: {classification.category}")
        print(f"   Subcategory: {classification.subcategory}")
        print(f"   Confidence: {classification.confidence:.2f}")
        
        print(f"   Reasoning:")
        for reason in classification.reasoning:
            print(f"     - {reason}")
        
        print(f"   Detected Keywords: {classification.detected_keywords[:8]}")
        
        # Test 2: Multi-label classification
        print("\n🏷️  2. Multi-Label Classification:")
        multi_categories = classifier.classify_multi_label(text      = text, 
                                                           threshold = 0.5,
                                                          )
        
        print(f"   Found {len(multi_categories)} relevant categories:")
        for i, category in enumerate(multi_categories):
            print(f"     {i+1}. {category.category} (confidence: {category.confidence:.2f})")
            if category.subcategory:
                print(f"        Subcategory: {category.subcategory}")
        
        # Test 3: Category descriptions
        print("\n📚 3. Available Categories:")
        all_categories = classifier.get_all_categories()
        print(f"   Total categories: {len(all_categories)}")
        
        # Show descriptions for top categories
        for category in multi_categories[:3]:
            description = classifier.get_category_description(category = category.category)
            print(f"     - {category.category}: {description}")
        
        return {'primary_classification' : classification,
                'multi_categories'       : multi_categories,
                'all_categories'         : all_categories,
               }
        
    except Exception as e:
        print(f"❌ Contract classification failed: {e}")
        print("   This may be due to model download requirements.")
        return None

# Run classification test
classification_results = test_contract_classification(text = document_text)



🏷️ STEP 5: Testing Contract Classifier
🎯 Initializing Contract Classifier...
✅ Models loaded successfully!

🔍 1. Single Category Classification:
   Primary Category: employment
   Subcategory: executive
   Confidence: 0.53
   Reasoning:
     - Strong keyword indicators for employment category (64% keyword match)
     - Specific subcategory identified: executive
     - Also contains elements of nda (secondary match: 0.52)
   Detected Keywords: ['employee', 'employment', 'job', 'position', 'salary', 'benefits', 'sick leave', 'probation']

🏷️  2. Multi-Label Classification:
   Found 2 relevant categories:
     1. employment (confidence: 0.53)
        Subcategory: executive
     2. nda (confidence: 0.52)
        Subcategory: mutual_nda

📚 3. Available Categories:
   Total categories: 12
     - employment: Employment agreements governing employer-employee relationships
     - nda: Non-disclosure and confidentiality agreements


## Complete Analysis Pipeline: Integrated Analysis Function

In [ ]:
print("\n🚀 STEP 6: Complete Analysis Pipeline")
print("=" * 60)

def complete_contract_analysis(file_path, use_ai=True):
    """
    Complete end-to-end contract analysis
    """
    print("🎯 Starting Complete Contract Analysis Pipeline...")
    
    # Initialize logging
    ContractAnalyzerLogger.setup(log_dir  = CONFIG["log_directory"], 
                                 app_name = "complete_analysis",
                                )
    
    analysis_results = {'file_info'      : {},
                        'validation'     : {},
                        'processing'     : {},
                        'classification' : {},
                        'llm_analysis'   : {},
                       }
    
    try:
        # Step 1: Document Reading
        print("\n📄 1. Document Reading...")
        reader                        = DocumentReader()
        file_contents                 = reader.read_file(file_path, "pdf")
        text                          = file_contents.get('text', '') if isinstance(file_contents, dict) else str(file_contents)
        
        analysis_results['file_info'] = {'text_length'        : len(text),
                                         'file_type'          : 'pdf',
                                         'extraction_success' : bool(text.strip()),
                                        }
        
        # Step 2: Contract Validation
        print("🔍 2. Contract Validation...")
        validator                          = ContractValidator()
        is_contract, val_type, val_message = validator.is_valid_contract(text)
        val_report                         = validator.get_validation_report(text)
        
        analysis_results['validation']     = {'is_contract'        : is_contract,
                                              'confidence_level'   : val_type,
                                              'validation_message' : val_message,
                                              'score'              : val_report['scores']['total'],
                                              'key_indicators'     : val_report['found_indicators'][:10],
                                             }
        
        # Step 3: Text Processing
        print("📝 3. Text Processing...")
        processor                      = TextProcessor(use_spacy = CONFIG["use_spacy"])
        stats                          = processor.get_text_statistics(text)
        entities                       = processor.extract_legal_entities(text)
        chunks                         = processor.chunk_text_for_embedding(text)
        
        analysis_results['processing'] = {'statistics'      : stats,
                                          'entity_counts'   : {k: len(v) for k, v in entities.items()},
                                          'key_entities'    : {'parties' : entities.get('parties', []),
                                                               'dates'   : entities.get('dates', []),
                                                               'amounts' : entities.get('amounts', []),
                                                              },
                                          'analysis_chunks' : len(chunks)
                                         }
        
        # Step 4: AI-Powered Analysis (Optional)
        if use_ai:
            print("🤖 4. AI-Powered Analysis...")
            try:
                # Contract Classification
                model_loader                       = ModelLoader()
                classifier                         = ContractClassifier(model_loader)
                classification                     = classifier.classify_contract(text)
                
                analysis_results['classification'] = {'primary_category' : classification.category,
                                                      'subcategory'      : classification.subcategory,
                                                      'confidence'       : classification.confidence,
                                                      'reasoning'        : classification.reasoning,
                                                     }
                
                # LLM Analysis
                llm_manager                        = LLMManager(default_provider = LLMProvider.OLLAMA)
                
                if LLMProvider.OLLAMA in llm_manager.get_available_providers():
                    summary_prompt                   = f"Provide a concise 3-bullet summary of this contract: {text}"
                    summary_response                 = llm_manager.complete(prompt      = summary_prompt,
                                                                            max_tokens  = 1024,
                                                                            temperature = 0.1,
                                                                           )
                    
                    analysis_results['llm_analysis'] = {'summary'  : summary_response.text if summary_response.success else "LLM analysis failed",
                                                        'provider' : 'ollama',
                                                       }
                
            except Exception as e:
                print(f"⚠️  AI analysis skipped: {e}")
                analysis_results['classification'] = {'error': str(e)}
                analysis_results['llm_analysis']   = {'error': str(e)}
        
        print("✅ Analysis completed successfully!")
        return analysis_results
        
    except Exception as e:
        print(f"❌ Analysis failed: {e}")
        return {'error': str(e)}

# Run complete analysis
print("🔧 Running complete analysis (this may take a few minutes)...")
complete_results = complete_contract_analysis(file_path = CONFIG["pdf_file_path"], 
                                              use_ai    = True,
                                             )

# Display results
print("\n" + "=" * 60)
print("📊 COMPLETE ANALYSIS RESULTS")
print("=" * 60)

if 'error' in complete_results:
    print(f"❌ Error: {complete_results['error']}")

else:
    # File Info
    file_info = complete_results['file_info']
    print(f"📄 FILE INFO:")
    print(f"   Text Length: {file_info['text_length']:,} characters")
    print(f"   Extraction Success: {file_info['extraction_success']}")
    
    # Validation
    validation = complete_results['validation']
    print(f"\n🔍 VALIDATION:")
    print(f"   Is Contract: {validation['is_contract']}")
    print(f"   Confidence: {validation['confidence_level']}")
    print(f"   Score: {validation['score']}")
    print(f"   Key Indicators: {', '.join(validation['key_indicators'][:3])}")
    
    # Processing
    processing = complete_results['processing']
    print(f"\n📝 PROCESSING:")
    print(f"   Sentences: {processing['statistics']['sentence_count']}")
    print(f"   Words: {processing['statistics']['word_count']}")
    print(f"   Language: {processing['statistics']['language']}")
    print(f"   Parties Found: {processing['entity_counts'].get('parties', 0)}")
    print(f"   Dates Found: {processing['entity_counts'].get('dates', 0)}")
    
    # Classification (if available)
    if complete_results['classification'] and 'primary_category' in complete_results['classification']:
        classification = complete_results['classification']
        print(f"\n🏷️  CLASSIFICATION:")
        print(f"   Category: {classification['primary_category']}")
        print(f"   Subcategory: {classification['subcategory']}")
        print(f"   Confidence: {classification['confidence']:.2f}")
    
    # LLM Analysis (if available)
    if complete_results['llm_analysis'] and 'summary' in complete_results['llm_analysis']:
        llm_analysis = complete_results['llm_analysis']
        print(f"\n💬 LLM ANALYSIS:")
        print(f"   Provider: {llm_analysis['provider']}")
        print(f"   Summary: {llm_analysis['summary']}")
        